In [ ]:
# Core data handling and computation
import numpy as np
import pandas as pd

# Financial data downloading
import yfinance as yf

# PyTorch for our model
import torch
import torch.nn as nn
import torch.nn.functional as F

# Date handling
from datetime import datetime, timedelta

# Type hints for better code clarity
from typing import List, Dict, Tuple, Optional

# Math operations
import math

In [ ]:
class HybridFinancialModel(nn.Module):
    def __init__(self,
                 state_dim,          # dimension of continuous state
                 memory_dim,         # dimension of memory/attention state
                 input_dim,          # dimension of market inputs
                 num_heads=8,        # number of attention heads
                 dropout=0.1):
        super().__init__()

        # State space components
        self.state_dynamics = StateEvolutionModule(
            state_dim=state_dim,
            memory_dim=memory_dim,
            input_dim=input_dim
        )

        # Attention components
        self.attention = MultiHeadAttention(
            state_dim=state_dim,
            memory_dim=memory_dim,
            num_heads=num_heads,
            dropout=dropout
        )

        # Coupling layers
        self.state_to_memory = StateProjModule(state_dim, memory_dim)
        self.memory_to_dynamics = MemoryProjModule(memory_dim, state_dim)

        # Output layers
        self.prediction_head = PredictionModule(state_dim, memory_dim)

    def forward(self, x, h=None, m=None, dt=0.1):
        batch_size = x.shape[0]

        # Initialize states if not provided
        if h is None:
            h = torch.zeros(batch_size, self.state_dim).to(x.device)
        if m is None:
            m = torch.zeros(batch_size, self.memory_dim).to(x.device)

        # Project current state to memory space
        memory_projection = self.state_to_memory(h)

        # Update memory through attention
        past_states = h  # In practice, you might want to maintain a longer history
        m_new = self.attention(h, past_states)

        # Project memory to state space
        state_influence = self.memory_to_dynamics(m_new, h)

        # Update state
        h_new = self.state_dynamics(h, state_influence, x, dt)

        # Generate predictions
        predictions = self.prediction_head(h_new, m_new)

        return predictions, h_new, m_new

class StateEvolutionModule(nn.Module):
    def __init__(self, state_dim, memory_dim, input_dim):
        super().__init__()
        # A(m) matrix - state transition conditioned on memory
        self.dynamics_net = nn.Sequential(
            nn.Linear(memory_dim, state_dim * state_dim),
            nn.Tanh()  # Ensure stability of dynamics
        )
        # B matrix - input effect
        self.input_proj = nn.Linear(input_dim, state_dim)

    def forward(self, h, m, x, dt):
        # Compute state transition matrix based on current memory
        A = self.dynamics_net(m).view(-1, self.state_dim, self.state_dim)

        # Continuous state update using Euler integration
        dh = torch.bmm(A, h.unsqueeze(-1)).squeeze(-1) + self.input_proj(x)
        h_next = h + dh * dt

        return h_next

class MultiHeadAttention(nn.Module):
    def __init__(self, state_dim, memory_dim, num_heads, dropout):
        super().__init__()
        # Standard multi-head attention with additional conditioning
        self.q_proj = nn.Linear(state_dim, memory_dim)
        self.k_proj = nn.Linear(state_dim, memory_dim)
        self.v_proj = nn.Linear(state_dim, memory_dim)

        self.num_heads = num_heads
        self.head_dim = memory_dim // num_heads

        self.dropout = nn.Dropout(dropout)

    def forward(self, h, past_states):
        # Project current state to Q/K/V space
        Q = self.q_proj(h).view(-1, self.num_heads, self.head_dim)
        K = self.k_proj(past_states).view(-1, self.num_heads, self.head_dim)
        V = self.v_proj(past_states).view(-1, self.num_heads, self.head_dim)

        # Compute attention scores
        scores = torch.bmm(Q, K.transpose(-2, -1)) / math.sqrt(self.head_dim)
        attn = F.softmax(scores, dim=-1)
        attn = self.dropout(attn)

        # Compute memory update
        m = torch.bmm(attn, V)
        return m.view(-1, self.num_heads * self.head_dim)

class StateProjModule(nn.Module):
    def __init__(self, state_dim, memory_dim):
        super().__init__()
        # We create a more sophisticated projection than just a linear layer
        # to allow the state to influence memory in a flexible way
        self.projection = nn.Sequential(
            nn.Linear(state_dim, state_dim * 2),
            nn.LayerNorm(state_dim * 2),
            nn.ReLU(),
            nn.Linear(state_dim * 2, memory_dim),
            nn.LayerNorm(memory_dim)
        )

    def forward(self, state):
        # Project the continuous state into the memory space
        # This allows the memory system to "understand" the current dynamics
        memory_projection = self.projection(state)
        return memory_projection

class MemoryProjModule(nn.Module):
    def __init__(self, memory_dim, state_dim):
        super().__init__()
        # Similar to StateProjModule, but going the other direction
        # We want the memory to be able to influence the state dynamics
        self.projection = nn.Sequential(
            nn.Linear(memory_dim, memory_dim * 2),
            nn.LayerNorm(memory_dim * 2),
            nn.ReLU(),
            nn.Linear(memory_dim * 2, state_dim),
            nn.LayerNorm(state_dim)
        )

        # Additional component to help regulate the influence of memory
        # This acts as a "gating" mechanism
        self.gate = nn.Sequential(
            nn.Linear(memory_dim + state_dim, state_dim),
            nn.Sigmoid()
        )

    def forward(self, memory, current_state):
        # Project memory into state space
        state_influence = self.projection(memory)

        # Compute a gate that determines how much the memory
        # should influence the state
        gate_input = torch.cat([memory, current_state], dim=-1)
        influence_gate = self.gate(gate_input)

        # Apply the gated influence
        gated_influence = state_influence * influence_gate
        return gated_influence

class PredictionModule(nn.Module):
    def __init__(self, state_dim, memory_dim):
        super().__init__()

        # Combined dimension of state and memory information
        combined_dim = state_dim + memory_dim

        # Main prediction network
        self.prediction_network = nn.Sequential(
            nn.Linear(combined_dim, combined_dim * 2),
            nn.LayerNorm(combined_dim * 2),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(combined_dim * 2, combined_dim),
            nn.LayerNorm(combined_dim),
            nn.ReLU(),
            nn.Dropout(0.1)
        )

        # Separate heads for different prediction tasks
        self.direction_head = nn.Linear(combined_dim, 1)  # Predicts price movement direction
        self.magnitude_head = nn.Linear(combined_dim, 1)  # Predicts size of movement
        self.uncertainty_head = nn.Linear(combined_dim, 1)  # Predicts prediction uncertainty

    def forward(self, state, memory):
        # Combine state and memory information
        combined_features = torch.cat([state, memory], dim=-1)

        # Process through main network
        processed_features = self.prediction_network(combined_features)

        # Generate predictions
        predictions = {
            'direction': self.direction_head(processed_features),  # Up/down prediction
            'magnitude': self.magnitude_head(processed_features),  # Expected size of move
            'uncertainty': torch.exp(self.uncertainty_head(processed_features))  # Uncertainty estimate
        }

        return predictions

Normal Market Dynamics

The first benchmark tests the model's ability to handle standard market behavior. We'll use the S&P 500 and its constituent stocks, focusing on daily data over the past 20 years

In [ ]:
class MarketDynamicsBenchmark:
    def __init__(self, start_date: datetime = "2004-01-01",
                 end_date: datetime = "2024-01-01"):
        # Load market data
        self.market_data = self._load_market_data()

        # Features we'll track
        self.features = [
            'close', 'volume', 'vwap', 'volatility_10d',
            'rsi_14', 'macd', 'bb_upper', 'bb_lower'
        ]

        # Performance metrics
        self.metrics = {
            'sharpe_ratio': self._calculate_sharpe,
            'sortino_ratio': self._calculate_sortino,
            'max_drawdown': self._calculate_drawdown,
            'hit_ratio': self._calculate_hit_ratio
        }

    def evaluate_model(self, model, test_period: Tuple[datetime, datetime]) -> Dict:
        """
        Evaluates model performance during normal market conditions
        Returns dictionary of performance metrics
        """
        # Implementation details here

In [ ]:
class RegimeChangeBenchmark:
    def __init__(self):
        # Define known regime change periods
        self.regime_changes = {
            '2008-09-15': 'Lehman Brothers Bankruptcy',
            '2020-03-23': 'COVID-19 Market Bottom',
            '2022-01-03': 'Fed Tightening Cycle'
        }

        # Metrics specific to regime changes
        self.transition_metrics = {
            'detection_lag': self._calculate_detection_lag,
            'false_positives': self._calculate_false_positives,
            'adaptation_speed': self._calculate_adaptation_speed
        }

    def evaluate_regime_handling(self, model) -> Dict:
        """
        Tests model's ability to:
        1. Detect regime changes
        2. Adapt to new market conditions
        3. Maintain performance during transitions
        """
        # Implementation details here

In [ ]:
class EventResponseBenchmark:
    def __init__(self):
        # Define different types of events to test
        self.event_types = {
            'earnings': self._load_earnings_events(),
            'fed_announcements': self._load_fed_events(),
            'economic_data': self._load_economic_events()
        }

        # Event-specific metrics
        self.event_metrics = {
            'prediction_accuracy': self._calculate_event_accuracy,
            'reaction_time': self._calculate_reaction_time,
            'profit_per_event': self._calculate_event_profit
        }

    def evaluate_event_handling(self, model, event_type: str) -> Dict:
        """
        Tests model's performance around specific event types
        """
        # Implementation details here

In [ ]:
class BaselineModels:
    def __init__(self):
        self.models = {
            'lstm': self._create_lstm_baseline(),
            'transformer': self._create_transformer_baseline(),
            'state_space': self._create_ssm_baseline(),
            'random_forest': self._create_rf_baseline()
        }

In [ ]:
class PracticalMetrics:
    def __init__(self):
        self.trading_metrics = {
            'latency': self._measure_inference_latency,
            'memory_usage': self._measure_memory_usage,
            'trade_execution_lag': self._measure_execution_lag,
            'transaction_costs': self._calculate_transaction_costs
        }

        self.risk_metrics = {
            'var_95': self._calculate_var,
            'expected_shortfall': self._calculate_es,
            'beta': self._calculate_beta,
            'correlation_stability': self._measure_correlation_stability
        }

In [ ]:
class MarketDataHandler:
    def __init__(self,
                 symbols: List[str],
                 lookback_period: int = 252):  # One trading year
        self.symbols = symbols
        self.lookback_period = lookback_period

        # Create data buffers for model features
        self.price_history = {}
        self.feature_buffers = {}

    def update_market_data(self, new_data: Dict):
        """
        Updates internal buffers with new market data
        Ensures we maintain proper history for feature calculation
        """
        for symbol in self.symbols:
            # Update price and volume data
            self.price_history[symbol] = self.price_history[symbol].append(
                new_data[symbol]
            ).tail(self.lookback_period)

            # Recalculate features
            self._update_features(symbol)

    def _update_features(self, symbol: str):
        """
        Calculates all necessary features for the model
        """
        data = self.price_history[symbol]

        # Calculate technical indicators
        self.feature_buffers[symbol] = {
            'returns': self._calculate_returns(data),
            'volatility': self._calculate_volatility(data),
            'volume_ma': self._calculate_volume_ma(data),
            # Add more features as needed
        }

In [ ]:
class TradingModel:
    def __init__(self,
                 model: HybridFinancialModel,
                 risk_limits: Dict[str, float]):
        self.model = model
        self.risk_limits = risk_limits
        self.current_state = None
        self.current_memory = None

    def predict(self,
                market_data: MarketDataHandler,
                current_positions: Dict[str, float]) -> Dict[str, float]:
        """
        Generates trading signals while respecting risk limits
        Returns: Dictionary of recommended position sizes
        """
        # Prepare features for model input
        features = self._prepare_features(market_data)

        # Get model predictions
        with torch.no_grad():
            self.current_state, self.current_memory = self.model(
                features,
                self.current_state,
                self.current_memory
            )

        # Convert predictions to position sizes
        positions = self._predictions_to_positions(
            self.current_state,
            current_positions,
            self.risk_limits
        )

        return positions

In [ ]:
class RiskManager:
    def __init__(self,
                 max_position_size: float,
                 max_portfolio_var: float,
                 max_leverage: float):
        self.max_position_size = max_position_size
        self.max_portfolio_var = max_portfolio_var
        self.max_leverage = max_leverage

    def validate_trades(self,
                       proposed_trades: Dict[str, float],
                       current_positions: Dict[str, float],
                       market_data: MarketDataHandler) -> Dict[str, float]:
        """
        Adjusts proposed trades to ensure they meet risk limits
        Returns: Dictionary of approved position changes
        """
        # Check position size limits
        adjusted_trades = self._check_position_limits(proposed_trades)

        # Check portfolio VaR
        adjusted_trades = self._check_portfolio_risk(
            adjusted_trades,
            current_positions,
            market_data
        )

        # Check leverage constraints
        final_trades = self._check_leverage(
            adjusted_trades,
            current_positions
        )

        return final_trades

In [ ]:
class ExecutionEngine:
    def __init__(self,
                 broker_api,
                 slippage_model: Optional[callable] = None):
        self.broker_api = broker_api
        self.slippage_model = slippage_model or self._default_slippage
        self.pending_orders = {}

    async def execute_trades(self,
                           trades: Dict[str, float],
                           market_data: MarketDataHandler):
        """
        Executes trades while managing transaction costs and market impact
        """
        for symbol, target_size in trades.items():
            # Calculate optimal trade size considering market impact
            trade_size = self._calculate_optimal_size(
                symbol,
                target_size,
                market_data
            )

            # Estimate transaction costs
            costs = self._estimate_transaction_costs(
                symbol,
                trade_size,
                market_data
            )

            if self._is_trade_profitable(trade_size, costs):
                await self._submit_order(symbol, trade_size)

In [ ]:
class PaperTradingEnvironment:
    def __init__(self,
                 initial_capital: float = 1_000_000,  # Start with $1M pseudo-capital
                 data_source: str = 'yahoo',  # Or your preferred data source
                 transaction_costs: bool = True,
                 market_impact: bool = True):

        self.capital = initial_capital
        self.positions = {}
        self.trade_history = []

        # Critical: Implement realistic market friction
        self.transaction_costs = {
            'commission': 0.001,  # 0.1% commission
            'slippage': 0.001,    # 0.1% assumed slippage
            'market_impact': self._calculate_market_impact if market_impact else None
        }

        # Important: Track all possible failure modes
        self.system_checks = {
            'data_delays': self._simulate_data_delays,
            'execution_fails': self._simulate_execution_failures,
            'price_gaps': self._simulate_price_gaps
        }

    def execute_trade(self, symbol: str, size: float, current_price: float):
        """
        Simulates trade execution with realistic constraints
        """
        # Simulate potential execution delay
        execution_delay = self._simulate_latency()

        # Get realistic fill price including market impact
        fill_price = self._calculate_fill_price(
            symbol, size, current_price, execution_delay
        )

        # Calculate total transaction costs
        costs = self._calculate_total_costs(size, fill_price)

        # Update capital and positions
        if self._can_execute_trade(size * fill_price + costs):
            self.capital -= (size * fill_price + costs)
            self.positions[symbol] = self.positions.get(symbol, 0) + size

            # Record trade details for analysis
            self._record_trade(symbol, size, fill_price, costs)

    def _calculate_fill_price(self, symbol: str, size: float,
                            current_price: float, delay: float) -> float:
        """
        Calculates realistic fill price including:
        - Price movement during execution delay
        - Market impact based on order size vs. average volume
        - Bid-ask spread
        """
        # Implementation details here

In [ ]:
def stress_test_strategy(strategy, environment):
    # Test during normal markets
    normal_market_results = run_test_period(
        strategy, environment,
        start_date='2019-01-01',
        end_date='2019-12-31'
    )

    # Test during high volatility
    crisis_results = run_test_period(
        strategy, environment,
        start_date='2020-03-01',
        end_date='2020-04-30'
    )

    # Test during low liquidity
    holiday_results = run_test_period(
        strategy, environment,
        start_date='2019-12-24',
        end_date='2019-12-26'
    )

In [ ]:
class StrategyMonitor:
    def __init__(self):
        self.metrics = {
            'sharpe_ratio': self._calculate_sharpe,
            'max_drawdown': self._calculate_drawdown,
            'win_rate': self._calculate_win_rate,
            'profit_factor': self._calculate_profit_factor,
            'recovery_time': self._calculate_recovery_time
        }

        # Critical: Track strategy degradation
        self.degradation_metrics = {
            'signal_decay': self._measure_signal_decay,
            'increased_correlation': self._measure_correlation_changes,
            'reduced_capacity': self._measure_capacity_constraints
        }

In [ ]:
def reality_check(strategy_results: Dict) -> bool:
    """
    Returns False if strategy shows suspicious performance
    """
    suspicious_patterns = [
        _check_unrealistic_sharpe(strategy_results),
        _check_perfect_timing(strategy_results),
        _check_impossible_fills(strategy_results),
        _check_capacity_constraints(strategy_results)
    ]

    return not any(suspicious_patterns)

In [ ]:
class ModelTrainer:
    def __init__(self,
                 model: HybridFinancialModel,
                 training_start: str = '2010-01-01',
                 training_end: str = '2022-12-31',
                 batch_size: int = 32,
                 sequence_length: int = 252):  # One trading year

        self.model = model
        self.batch_size = batch_size
        self.sequence_length = sequence_length

        # Initialize data loaders
        self.data_handler = FinancialDataHandler(
            training_start=training_start,
            training_end=training_end
        )


    def create_data_loader(self, features: torch.Tensor, labels: torch.Tensor) -> torch.utils.data.DataLoader:
        """
        Creates a DataLoader that efficiently feeds data to our model during training.

        The DataLoader handles:
        1. Batching the data into consistent sizes
        2. Shuffling the data (while maintaining sequence order within each sample)
        3. Loading data efficiently in parallel

        Args:
            features: Tensor of shape (num_samples, sequence_length, num_features)
            labels: Tensor of shape (num_samples, num_prediction_targets)

        Returns:
            DataLoader that yields batches of (features, labels)
        """
        # Create a TensorDataset that pairs features with labels
        dataset = torch.utils.data.TensorDataset(features, labels)

        # Create the DataLoader with appropriate settings
        data_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=self.batch_size,
            shuffle=True,  # Shuffle data between epochs
            num_workers=2,  # Use 2 worker processes for loading data
            pin_memory=True,  # Speed up data transfer to GPU if used
            drop_last=True  # Drop the last incomplete batch if any
        )

        return data_loader

    def prepare_training_data(self):
        """
        Prepares and stores training data, splitting it into training and validation sets.
        This method processes our raw financial data into a format suitable for training
        our hybrid model.
        """
        # Get features and labels from our data handler
        features, labels = self.data_handler.prepare_training_data(self.sequence_length)

        # Split into training and validation sets (80-20 split)
        # We use a time-based split since this is financial data
        split_idx = int(0.8 * len(features))

        # Store the split data as class attributes
        self.train_data = {
            'features': features[:split_idx],
            'labels': labels[:split_idx]
        }

        self.val_data = {
            'features': features[split_idx:],
            'labels': labels[split_idx:]
        }

        print(f"Training data shape: {self.train_data['features'].shape}")
        print(f"Validation data shape: {self.val_data['features'].shape}")



    def train_model(self, epochs: int = 100, learning_rate: float = 1e-4):
        """
        Trains the hybrid model using our prepared training and validation data.
        """
        # Create data loaders using our stored data
        train_loader = self.create_data_loader(
            self.train_data['features'],
            self.train_data['labels']
        )

        val_loader = self.create_data_loader(
            self.val_data['features'],
            self.val_data['labels']
        )

        for epoch in range(epochs):
            # Training loop
            self.model.train()
            for batch in train_loader:
                optimizer.zero_grad()

                # Forward pass
                state_pred, memory = self.model(batch['features'])

                # Calculate loss
                loss = loss_fn(
                    state_pred=state_pred,
                    memory=memory,
                    targets=batch['labels']
                )

                # Backward pass
                loss.backward()
                optimizer.step()

            # Validation phase
            self.model.eval()
            val_metrics = self.validate_epoch(val_loader)

            # Early stopping check
            if self.should_stop_early(val_metrics):
                break

    def create_loss_function(self):
        """
        Creates a composite loss function appropriate for financial data
        """
        def custom_loss(state_pred, memory, targets):
            # Directional accuracy loss
            direction_loss = F.binary_cross_entropy_with_logits(
                state_pred['direction'],
                targets['direction']
            )

            # Magnitude loss with asymmetric penalties
            magnitude_loss = self.asymmetric_magnitude_loss(
                state_pred['magnitude'],
                targets['magnitude']
            )

            # Temporal coherence loss for state evolution
            coherence_loss = self.temporal_coherence_loss(state_pred)

            return (
                0.4 * direction_loss +
                0.4 * magnitude_loss +
                0.2 * coherence_loss
            )

        return custom_loss

In [ ]:
class FinancialDataHandler:
    def __init__(self,
                 training_start: str,
                 training_end: str,
                 symbols: List[str] = ['SPY'],  # Default to S&P 500 ETF
                 data_source: str = 'yahoo'):

        self.training_start = training_start
        self.training_end = training_end
        self.symbols = symbols
        self.data_source = data_source

        # Initialize feature calculators
        self.technical_features = [
            self._calculate_returns,
            self._calculate_volatility,
            self._calculate_volume_features,
            self._calculate_price_patterns
        ]

        self.market_features = [
            self._calculate_market_regime,
            self._calculate_correlation_structure,
            self._calculate_liquidity_metrics
        ]

        # Load and prepare initial data
        self.data = self._load_market_data()

    def _load_market_data(self) -> pd.DataFrame:
        """
        Loads market data for the specified period and symbols
        """
        data = {}
        for symbol in self.symbols:
            # Using yfinance to load data
            data[symbol] = yf.download(
                symbol,
                start=self.training_start,
                end=self.training_end
            )

        # Combine all symbols into one DataFrame
        combined_data = pd.concat(data.values(), axis=1, keys=data.keys())
        return combined_data

    def _calculate_returns(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Calculates various return-based features
        """
        features = pd.DataFrame(index=data.index)

        # Daily returns
        features['returns'] = data['Close'].pct_change()

        # Rolling returns for different periods
        for window in [5, 10, 21]:  # 1 week, 2 weeks, 1 month
            features[f'returns_{window}d'] = data['Close'].pct_change(window)

        return features

    def _calculate_volatility(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Calculates volatility-based features
        """
        features = pd.DataFrame(index=data.index)

        # Rolling volatility for different periods
        for window in [10, 21, 63]:  # 2 weeks, 1 month, 3 months
            returns = data['Close'].pct_change()
            features[f'volatility_{window}d'] = returns.rolling(window).std()

        return features

    def _calculate_volume_features(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Calculates volume-based features
        """
        features = pd.DataFrame(index=data.index)

        # Volume changes
        features['volume_change'] = data['Volume'].pct_change()

        # Rolling average volume
        for window in [5, 10, 21]:
            features[f'volume_ma_{window}d'] = data['Volume'].rolling(window).mean()

        # Volume relative to moving average
        features['volume_rel_ma'] = data['Volume'] / features['volume_ma_21d']

        return features

    def _calculate_price_patterns(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Calculates technical price patterns and indicators
        Returns DataFrame with price-based technical indicators
        """
        features = pd.DataFrame(index=data.index)

        # Moving averages for different timeframes
        for window in [5, 10, 20, 50]:
            features[f'sma_{window}d'] = data['Close'].rolling(window=window).mean()

        # Price relative to moving averages (momentum indicators)
        features['price_sma_ratio_20d'] = data['Close'] / features['sma_20d']

        # RSI (Relative Strength Index)
        delta = data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
        rs = gain / loss
        features['rsi_14d'] = 100 - (100 / (1 + rs))

        # Bollinger Bands
        sma_20 = data['Close'].rolling(window=20).mean()
        std_20 = data['Close'].rolling(window=20).std()
        features['bb_upper'] = sma_20 + (std_20 * 2)
        features['bb_lower'] = sma_20 - (std_20 * 2)
        features['bb_position'] = (data['Close'] - features['bb_lower']) / (features['bb_upper'] - features['bb_lower'])

        # MACD (Moving Average Convergence Divergence)
        exp1 = data['Close'].ewm(span=12, adjust=False).mean()
        exp2 = data['Close'].ewm(span=26, adjust=False).mean()
        features['macd'] = exp1 - exp2
        features['macd_signal'] = features['macd'].ewm(span=9, adjust=False).mean()

        # Handle any NaN values created by the rolling windows
        features = features.fillna(method='bfill')

        return features

    def _calculate_market_regime(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Identifies market regimes using various metrics.
        Market regimes help us understand the broader context of market behavior,
        such as whether we're in a bull market, bear market, or high volatility period.
        """
        features = pd.DataFrame(index=data.index)

        # First, let's calculate some trend indicators
        # We'll use multiple timeframes to capture different regime aspects
        returns = data['Close'].pct_change()

        # Trend strength using moving averages
        sma_50 = data['Close'].rolling(window=50).mean()
        sma_200 = data['Close'].rolling(window=200).mean()

        # Golden/Death Cross indicator
        features['trend_regime'] = (sma_50 > sma_200).astype(float)

        # Volatility regime using realized volatility
        # We compare short-term vs long-term volatility
        vol_20 = returns.rolling(window=20).std()
        vol_60 = returns.rolling(window=60).std()
        features['volatility_regime'] = (vol_20 > vol_60).astype(float)

        # Market momentum regime
        # Using 3-month and 12-month returns
        features['momentum_regime'] = (data['Close'].pct_change(63) > 0).astype(float)

        # Composite regime indicator combining all aspects
        features['composite_regime'] = (
            features['trend_regime'] +
            features['volatility_regime'] +
            features['momentum_regime']
        ) / 3.0

        # Handle any NaN values from the rolling calculations
        features = features.fillna(method='bfill')

        return features

    def _calculate_correlation_structure(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Analyzes the correlation structure of the market. This helps us understand
        how different market components are interacting with each other, which
        can signal important regime changes or risk conditions.
        """
        features = pd.DataFrame(index=data.index)

        # Calculate returns for our correlation analysis
        returns = data['Close'].pct_change()

        # Rolling correlation with market (if we have multiple symbols)
        if len(self.symbols) > 1:
            market_returns = returns[self.symbols[0]]  # Assuming first symbol is market
            for symbol in self.symbols[1:]:
                # 20-day rolling correlation with market
                features[f'market_corr_{symbol}'] = returns[symbol].rolling(20).corr(market_returns)

        # Correlation stability - how stable are correlations over time?
        # We calculate this by looking at the standard deviation of correlations
        window_sizes = [20, 60]  # Look at both month and quarter timeframes

        for window in window_sizes:
            # Calculate rolling mean of absolute returns (a measure of volatility)
            vol = returns.abs().rolling(window=window).mean()

            # Calculate how this volatility measure correlates with returns
            # This helps identify if we're in a risk-on/risk-off regime
            features[f'vol_ret_corr_{window}d'] = (
                returns.rolling(window=window)
                .corr(vol)
            )

            # Dispersion of returns - helps identify if market is moving as one
            if len(self.symbols) > 1:
                features[f'return_dispersion_{window}d'] = (
                    returns.std(axis=1)
                    .rolling(window=window)
                    .mean()
                )

        # Risk-on/Risk-off indicator using correlation between
        # high volatility and high correlation periods
        features['risk_regime'] = (
            features['vol_ret_corr_20d']
            .rolling(window=10)
            .mean()
        )

        # Handle any NaN values from the rolling calculations
        features = features.fillna(method='bfill')

        return features

    def _calculate_liquidity_metrics(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Calculates various liquidity metrics to understand how easily we can trade
        without significantly impacting price. Liquidity is crucial for understanding
        transaction costs and execution risk.
        """
        features = pd.DataFrame(index=data.index)

        # First, let's calculate basic volume-based liquidity measures
        typical_price = (data['High'] + data['Low'] + data['Close']) / 3

        # Dollar volume - a fundamental measure of trading activity
        features['dollar_volume'] = typical_price * data['Volume']

        # Amihud illiquidity ratio
        # This measures price impact per dollar of trading volume
        # Higher values indicate lower liquidity
        daily_returns = data['Close'].pct_change().abs()
        features['amihud_ratio'] = (daily_returns / features['dollar_volume']) * 1e6

        # Relative strength of volume
        # Compare current volume to recent history
        for window in [5, 20]:
            vol_ma = data['Volume'].rolling(window=window).mean()
            features[f'volume_strength_{window}d'] = data['Volume'] / vol_ma

        # Bid-ask spread proxy using high-low range
        # Since we might not have actual bid-ask data
        features['hl_spread'] = (data['High'] - data['Low']) / typical_price

        # Volume volatility
        # High volume volatility can indicate unstable liquidity conditions
        for window in [5, 20]:
            features[f'volume_volatility_{window}d'] = (
                data['Volume']
                .rolling(window=window)
                .std() / data['Volume'].rolling(window=window).mean()
            )

        # Flow toxicity measure
        # Adapted from VPIN (Volume-synchronized Probability of Informed Trading)
        # High values might indicate informed trading and potential liquidity problems
        signed_volume = data['Volume'] * np.sign(data['Close'] - data['Open'])
        features['flow_toxicity'] = (
            signed_volume
            .rolling(window=20)
            .sum()
            .abs() / data['Volume'].rolling(window=20).sum()
        )

        # Liquidity resilience
        # How quickly volume returns after a high-volume day
        high_volume_days = data['Volume'] > data['Volume'].rolling(window=20).mean() * 1.5
        volume_after_shock = (
            data['Volume']
            .shift(-1)  # Next day's volume
            .rolling(window=5)
            .mean()
        )
        features['liquidity_resilience'] = volume_after_shock / data['Volume']

        # Market efficiency coefficient
        # Helps identify if price changes are orderly or disorderly
        for window in [5, 20]:
            log_prices = np.log(data['Close'])
            variance_ratio = (
                log_prices.diff(window).var()
                / (window * log_prices.diff().var())
            )
            features[f'market_efficiency_{window}d'] = variance_ratio

        # Handle any NaN values from the calculations
        features = features.fillna(method='bfill')

        return features


    def prepare_training_data(self, sequence_length: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Prepares features and labels for training
        """
        # Calculate all features
        features = self._calculate_features(self.data)

        # Create labels (future returns)
        labels = self._create_labels(self.data)

        # Convert to tensors and create sequences
        X, y = self._create_sequences(features, labels, sequence_length)

        return X, y

    def _create_sequences(self,
                         features: pd.DataFrame,
                         labels: pd.DataFrame,
                         sequence_length: int) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Creates sequences for training
        """
        X, y = [], []

        for i in range(len(features) - sequence_length):
            X.append(features.iloc[i:i+sequence_length].values)
            y.append(labels.iloc[i+sequence_length])

        return torch.FloatTensor(X), torch.FloatTensor(y)

    def _create_labels(self, data: pd.DataFrame) -> pd.DataFrame:
        """
        Creates labels for training
        """
        labels = pd.DataFrame(index=data.index)

        # Future returns
        future_returns = data['Close'].pct_change().shift(-1)

        # Create binary direction label
        labels['direction'] = (future_returns > 0).astype(float)

        # Create magnitude label
        labels['magnitude'] = future_returns.abs()

        return labels

In [ ]:
# Initialize the model with basic parameters
model = HybridFinancialModel(
    state_dim=32,      # Size of continuous state
    memory_dim=64,     # Size of attention/memory state
    input_dim=20,      # Number of input features
    num_heads=4        # Start with fewer attention heads
)

# Create trainer
trainer = ModelTrainer(
    model=model,
    training_start='2010-01-01',
    training_end='2022-12-31'
)

# Train model
trainer.train_model(
    epochs=100,
    learning_rate=1e-4
)

[*********************100%***********************]  1 of 1 completed


AttributeError: 'ModelTrainer' object has no attribute 'train_data'

In [ ]:
# Create validation environment
paper_trading = PaperTradingEnvironment(
    initial_capital=1_000_000,
    transaction_costs=True
)

# Run validation
validation_results = paper_trading.run_validation(
    model=model,
    start_date='2023-01-01',
    end_date='2023-12-31'
)

# Analyze results
analysis = StrategyAnalyzer(validation_results)
analysis.print_summary()